## CNN으로 MNIST 데이터 분석

### 딥러닝 학습 단계 
1. 라이브러리 호출
2. GPU 사용호출
3. 학습에 사용되는 parameter 설정
4. 데이터 셋을 가져오고 loader 만들기 
5. 학습 모델 만들기 (class CNN)
6. Loss function (Criterion)을 선택, optimizer 선택
7. train, loss check 
8. 성능 확인 

### CNN 구조 

* layer1
    * `Convolution layer = (in_c=1,out_c=32, kernel_size=3, stride=1, padding =1)`
    * `MaxPool layer = (kernel_size=2, stride=2)`
* layer2 
    * `Convolution layer = (in_c=32, out_c=64, kernel_size=3, stride=1, padding=1 )`
    * `MaxPool layer = (kernel_size=2, stride=2)`

* view => (batch_size x [7,7,64] => batch_size x[3136])
* Fully_connect layer => (input=3136, output=10)

### loss
* Cross Entropy Loss
    * SoftMax
    * NLL loss

In [1]:
# Lab 11 MNIST and Convolutional Neural Network
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [16]:
import torch.nn as nn
# CNN example
inputs = torch.Tensor(1,1,28,28) # batch =1, channel=1, width,height=28
print(inputs.shape)
# maxpooling과 conv만 사용, relu는 예시에서는 생략
con1 = nn.Conv2d(1,32,3,padding=1)
pool = nn.MaxPool2d(2) # 반 x 반 
con2 = nn.Conv2d(32,64,3,padding=1)
print(con1)
print(pool)
print(con2)

torch.Size([1, 1, 28, 28])
Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [22]:
out = con1(inputs)
print(out.shape)
out = pool(out)
print(out.shape)
out = con2(out)
print(out.shape)
out = pool(out)
print(out.shape)
out = out.view(out.size(0),-1)
print(out.shape)
fc = nn.Linear(3136,10)
out = fc(out)
print(out.shape)

torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])
torch.Size([1, 3136])
torch.Size([1, 10])


In [2]:
# gpu 사용
device = 'cuda' if torch.cuda.is_available() else 'cpu'
 
# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out


In [7]:
# instantiate CNN model
model = CNN().to(device)

In [8]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:

# train my model
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.220258787
[Epoch:    2] cost = 0.0608599558
[Epoch:    3] cost = 0.0459129922
[Epoch:    4] cost = 0.0366976671
[Epoch:    5] cost = 0.0299328379
[Epoch:    6] cost = 0.0262569282
[Epoch:    7] cost = 0.0208396725
[Epoch:    8] cost = 0.0185660906
[Epoch:    9] cost = 0.0157809369
[Epoch:   10] cost = 0.013422614
[Epoch:   11] cost = 0.0117681315
[Epoch:   12] cost = 0.0086277714
[Epoch:   13] cost = 0.00779873133
[Epoch:   14] cost = 0.00714771263
[Epoch:   15] cost = 0.00617111241
Learning Finished!


In [10]:
# Test model and check accuracy
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/Users/kkyuhun/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/kkyuhun/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9854000210762024
